## Python绘制时间线轮播多图

In [1]:
import pandas as pd
from pyecharts import options as opts
from pyecharts.charts import Pie, Bar, Timeline

### 1. 读取北京2019年天气数据

In [2]:
df = pd.read_csv("./datas/beijing_tianqi/beijing_tianqi_2019.csv")
df.head(3)

,ymd,bWendu,yWendu,tianqi,fengxiang,fengli,aqi,aqiInfo,aqiLevel
0,2019-01-01,1℃,-10℃,晴~多云,西北风,1级,56,良,2
1,2019-01-02,1℃,-9℃,多云,东北风,1级,60,良,2
2,2019-01-03,2℃,-7℃,霾,东北风,1级,165,中度污染,4


In [3]:
df["month"] = pd.to_datetime(df["ymd"]).dt.month
df.head(3)

,ymd,bWendu,yWendu,tianqi,fengxiang,fengli,aqi,aqiInfo,aqiLevel,month
0,2019-01-01,1℃,-10℃,晴~多云,西北风,1级,56,良,2,1
1,2019-01-02,1℃,-9℃,多云,东北风,1级,60,良,2,1
2,2019-01-03,2℃,-7℃,霾,东北风,1级,165,中度污染,4,1


In [4]:
# 统计每个月份的每种天气出现次数
df_agg = df.groupby(["month", "tianqi"]).size().reset_index()
df_agg.columns = ["month", "tianqi", "count"]
df_agg.head(10)

,month,tianqi,count
0,1,多云,9
1,1,多云~晴,1
2,1,晴,12
3,1,晴~多云,4
4,1,霾,3
5,1,霾~多云,1
6,1,霾~晴,1
7,2,多云,9
8,2,多云~晴,3
9,2,小雪~多云,2


In [5]:
# 怎样算出1月份的天气次数排名
df_agg[df_agg["month"]==1][["tianqi", "count"]].sort_values(by="count", ascending=False).values.tolist()

[['晴', 12],
 ['多云', 9],
 ['晴~多云', 4],
 ['霾', 3],
 ['多云~晴', 1],
 ['霾~多云', 1],
 ['霾~晴', 1]]

### 2. 按月变化-天气频率柱状图

In [6]:
timeline = Timeline()
timeline.add_schema(play_interval=1000)
for month in df_agg["month"].unique():
    data = (
        df_agg[df_agg["month"]==month][["tianqi", "count"]]
        .sort_values(by="count", ascending=True)
        .values.tolist()
    )
    
    # 绘制柱状图
    bar = Bar()
   
    # x轴是天气名称
    bar.add_xaxis([x[0] for x in data])
    # y轴是出现次数
    bar.add_yaxis("", [x[1] for x in data])
    # 让柱状图横放
    bar.reversal_axis()
    bar.set_series_opts(label_opts=opts.LabelOpts(position="right"))
    bar.set_global_opts(title_opts=opts.TitleOpts(title="北京每月天气变化"))
    
    timeline.add(bar, f"{month}月")
    
timeline.render_notebook()